In [ ]:
# !pip install imutils

In [ ]:
import sys
import os
import cv2
import datetime
# import imutils
import numpy as np
from pathlib import Path
import random
import xml.etree.ElementTree as ET

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer

import pickle

In [ ]:
data = []
labels = []
bboxes = []
imagePaths = []

# classes = ["trafficlight", "speedlimit", "crosswalk", "stop"]

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip -q "/content/gdrive/My Drive/signboarddetection.zip"

In [ ]:
annot_dir  = "annotations"
images_dir = "images"  #signboarddetection/

In [ ]:
for filename in os.listdir(annot_dir):
    f = os.path.join(annot_dir, filename)
    tree = ET.parse(f)
    root = tree.getroot()

    w = int(root.find('.//size/width').text)
    h = int(root.find('.//size/height').text)

    for box in root.findall('.//bndbox'):
        xmin = int(box.find('xmin').text) / w
        ymin = int(box.find('ymin').text) / h
        xmax = int(box.find('xmax').text) / w
        ymax = int(box.find('ymax').text) / h

    label = root.find('.//object/name').text

    imname = root.find('.//filename').text
    impath = os.path.join(images_dir, imname)
    image = load_img(impath, target_size=(224,224))
    image = img_to_array(image)

    data.append(image)
    labels.append(label)
    bboxes.append((xmin,ymin,xmax,ymax))
    imagePaths.append(impath)

In [ ]:
imgplot = plt.imshow(image.astype('uint8'))
print(labels[-1])
print(bboxes[-1])
print(imagePaths[-1])
plt.show()

In [ ]:
# normalize -> from [0-255] to [0-1]
data = np.array(data, dtype="float32") / 255.0

# convert to np arrays
labels = np.array(labels)
bboxes = np.array(bboxes, dtype="float32")
imagePaths = np.array(imagePaths)

# one-hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
# test-train split 20%,80%

split = train_test_split(data,labels,bboxes,imagePaths,test_size=0.20,random_state=1)

In [ ]:
split

In [ ]:
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]
(trainPaths,  testPaths)  = split[6:]

# saving test files for later use
with open("testing_multiclass.txt", "w") as f:
    f.write("\n".join(testPaths))

In [ ]:
testLabels

### The Neural Net : Architecture

In [ ]:
vgg = VGG16(weights="imagenet",
            include_top=False,
            input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
# freeze training any of the layers of VGGNet
vgg.trainable = False

# max-pooling is output of VGG, flattening it further
flatten = vgg.output
flatten = Flatten()(flatten)

In [ ]:
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid", name="bounding_box")(bboxHead)
# 4 neurons correspond to 4 co-ords in output bbox

In [ ]:
softmaxHead = Dense(512, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(len(lb.classes_), activation="softmax", name="class_label")(softmaxHead)

In [ ]:
model = Model(
    inputs=vgg.input,
    outputs=(bboxHead, softmaxHead))

In [ ]:
INIT_LR = 1e-4
NUM_EPOCHS = 11
BATCH_SIZE = 16

In [ ]:
losses = {
    "class_label": "categorical_crossentropy",
    "bounding_box": "mean_squared_error",
}

lossWeights = {
    "class_label": 1.0,
    "bounding_box": 1.0
}

trainTargets = {
    "class_label": trainLabels,
    "bounding_box": trainBBoxes
}

testTargets = {
    "class_label": testLabels,
    "bounding_box": testBBoxes
}

In [ ]:
opt = Adam(INIT_LR)

model.compile(loss=losses,
              optimizer=opt,
              metrics=["accuracy"],
              loss_weights=lossWeights)

print(model.summary())

### Training

In [ ]:
H = model.fit(
    trainImages, trainTargets,
    validation_data=(testImages, testTargets),
    batch_size=BATCH_SIZE,


    epochs=NUM_EPOCHS,
    verbose=1)

In [ ]:
model.save("model_bbox_regression_and_classification", save_format="h5")


In [ ]:
f = open("lb.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()

In [ ]:
lossNames = ["loss",
             "class_label_loss",
             "bounding_box_loss"]

N = np.arange(0, NUM_EPOCHS)
plt.style.use("ggplot")
(fig, ax) = plt.subplots(3, 1, figsize=(17, 25))

# plotting the loss for training and validation data

for (i, l) in enumerate(lossNames):
    title = "Loss for {}".format(l) if l != "loss" else "Total loss"
    ax[i].set_title(title)
    ax[i].set_xlabel("Epoch #")
    ax[i].set_ylabel("Loss")
    ax[i].plot(N, H.history[l], label=l)
    ax[i].plot(N, H.history["val_" + l], label="val_" + l)
    ax[i].legend()

In [ ]:
# create a new figure for the accuracies
plt.style.use("ggplot")
plt.figure(figsize=(17, 10))

plt.plot(N, H.history["class_label_accuracy"],
         label="class_label_train_acc")
plt.plot(N, H.history["val_class_label_accuracy"],
         label="val_class_label_acc")

plt.title("Class Label Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")

# save the accuracies plot
plotPath = os.path.sep.join(["acc_label.png"])
plt.savefig(plotPath)

In [ ]:
# create a new figure for the accuracies
plt.style.use("ggplot")
plt.figure(figsize=(17, 10))

plt.plot(N, H.history["bounding_box_accuracy"],
         label="Training Data")
plt.plot(N, H.history["val_bounding_box_accuracy"],
         label="Testing Data")

plt.title("Bounding Box Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")

# save the accuracies plot
plotPath = os.path.sep.join(["acc_bbox.png"])
plt.savefig(plotPath)

### Testing the model

In [ ]:
path = "testing_multiclass.txt"
filenames = open(path).read().strip().split("\n")
imagePaths = []

for f in filenames:
    imagePaths.append(f)

In [ ]:
model = load_model("./model_bbox_regression_and_classification")
lb = pickle.loads(open("./lb.pickle", "rb").read())


In [ ]:
actual_labels = []
predicted_labels = []

for imagePath in imagePaths:

    # loading input image
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)

    # predicting bbox and label
    (boxPreds, labelPreds) = model.predict(image)
    (startX, startY, endX, endY) = boxPreds[0]

    # finding class label with highest pred. probability
    i = np.argmax(labelPreds, axis=1)
    label = lb.classes_[i][0]
    if (label == "trafficlight"):
      predicted_labels.append([1,0,0,0])
    if (label == "speedlimit"):
      predicted_labels.append([0,1,0,0])
    if (label == "crosswalk"):
      predicted_labels.append([0,0,1,0])
    if (label == "stop"):
      predicted_labels.append([0,0,0,1])

    # loading the image in OpenCV format
    image = cv2.imread(imagePath)
    # image = imutils.resize(image, width=600)
    (h, w) = image.shape[:2]

    # scaling pred. bbox coords according to image dims
    startX = int(startX * w)
    startY = int(startY * h)
    endX = int(endX * w)
    endY = int(endY * h)

    # drawing bbox and label on image
    y = startY - 10 if startY - 10 > 10 else startY + 10

    cv2.putText(image,
                label,
                (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 0),
                2)

    cv2.rectangle(image,
                  (startX, startY),
                  (endX, endY),
                  (0, 255, 0),
                  2)

    # showing the output image
    imgplot = plt.imshow(
        cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype('uint8'))
    plt.show()

In [ ]:
predicted_labels

In [ ]:
testLabels

In [ ]:
predicted_labels_simple = []
for arr in (predicted_labels):
  if (arr[0]==1):
    predicted_labels_simple.append(0)
  if (arr[1]==1):
    predicted_labels_simple.append(1)
  if (arr[2]==1):
    predicted_labels_simple.append(2)
  if (arr[3]==1):
    predicted_labels_simple.append(3)

test_labels_simple = []
for arr in (testLabels):
  if (arr[0]==1):
    test_labels_simple.append(0)
  if (arr[1]==1):
    test_labels_simple.append(1)
  if (arr[2]==1):
    test_labels_simple.append(2)
  if (arr[3]==1):
    test_labels_simple.append(3)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import torch

In [ ]:
classes = ["trafficlight", "speedlimit", "crosswalk", "stop"]

In [ ]:
confusion = confusion_matrix(test_labels_simple, predicted_labels_simple)

NameError: ignored

In [ ]:
confusion

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(confusion,annot=True,cmap='Blues')  #,annot=classes,fmt="",cmap='RdYlGn',linewidths=0.30,ax=ax)